In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from typing import Tuple, Optional
import warnings
import json

def ecfp_cluster_train_val_split(
    df: pd.DataFrame,
    val_ratio: float = 0.2,
    smiles_col: str = 'smiles',
    mol_col: Optional[str] = 'rdmol_confs5',
    ecfp_radius: int = 2,
    ecfp_bits: int = 2048,
    n_clusters: Optional[int] = None,
    random_state: int = 42
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    ECFP 기반 클러스터링을 통해 train/validation set을 분리하는 함수
    
    Parameters:
    -----------
    df : pd.DataFrame
        분리할 데이터프레임
    val_ratio : float, default=0.2
        validation set 비율 (0~1)
    smiles_col : str, default='smiles'
        SMILES 문자열이 있는 컬럼명
    mol_col : str, optional, default='rdmol_confs5'
        RDKit mol 객체가 있는 컬럼명 (None이면 SMILES에서 생성)
    ecfp_radius : int, default=2
        ECFP의 반지름
    ecfp_bits : int, default=2048
        ECFP의 비트 수
    n_clusters : int, optional
        클러스터 수 (None이면 자동 계산)
    random_state : int, default=42
        랜덤 시드
        
    Returns:
    --------
    Tuple[pd.DataFrame, pd.DataFrame]
        (train_df, val_df) 튜플
    """
    
    df_copy = df.copy()
    
    # 1. ECFP 피처 생성
    print("ECFP 피처 생성 중...")

    morgan_gen = GetMorganGenerator(radius=ecfp_radius, fpSize=ecfp_bits)

    ecfp_features = []
    valid_indices = []
    
    for idx, row in df_copy.iterrows():
        try:
            # mol 객체가 있으면 사용, 없으면 SMILES에서 생성
            if mol_col and mol_col in df_copy.columns and pd.notna(row[mol_col]):
                mol = row[mol_col]
            else:
                mol = Chem.MolFromSmiles(row[smiles_col])
            
            if mol is not None:
                # ECFP 생성
                fp = morgan_gen.GetFingerprint(mol)
                ecfp_features.append(np.array(fp))
                valid_indices.append(idx)
            else:
                print(f"Warning: 인덱스 {idx}에서 분자 생성 실패")
                
        except Exception as e:
            print(f"Warning: 인덱스 {idx}에서 오류 발생: {e}")
            continue
    
    if len(ecfp_features) == 0:
        raise ValueError("유효한 ECFP 피처를 생성할 수 없습니다.")
    
    # 유효한 데이터만 필터링
    df_valid = df_copy.loc[valid_indices].reset_index(drop=True)
    ecfp_matrix = np.array(ecfp_features)
    
    print(f"유효한 분자 수: {len(ecfp_features)}")
    
    # 2. 클러스터 수 결정
    if n_clusters is None:
        # 적절한 클러스터 수 자동 계산 (데이터 크기의 제곱근 정도)
        n_clusters = max(2, int(np.sqrt(len(ecfp_features)) / 2))
        # validation 비율을 맞추기 위해 조정
        n_clusters = min(n_clusters, int(1 / val_ratio))
    
    print(f"클러스터 수: {n_clusters}")
    
    # 3. K-means 클러스터링
    print("클러스터링 수행 중...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
    cluster_labels = kmeans.fit_predict(ecfp_matrix)
    
    # 4. 클러스터별 크기 계산
    unique_clusters, cluster_counts = np.unique(cluster_labels, return_counts=True)
    cluster_info = list(zip(unique_clusters, cluster_counts))
    cluster_info.sort(key=lambda x: x[1], reverse=True)  # 크기 순 정렬
    
    print("클러스터 정보:")
    for cluster_id, count in cluster_info:
        print(f"  클러스터 {cluster_id}: {count}개 분자")
    
    # 5. validation 클러스터 선택
    # 목표 validation 크기
    target_val_size = int(len(df_valid) * val_ratio)
    
    # 클러스터를 하나씩 validation에 할당하면서 목표 크기에 근접하도록
    val_clusters = []
    current_val_size = 0
    
    for cluster_id, count in cluster_info:
        if current_val_size + count <= target_val_size * 1.2:  # 20% 여유
            val_clusters.append(cluster_id)
            current_val_size += count
        if current_val_size >= target_val_size * 0.8:  # 80% 이상이면 중단
            break
    
    # 6. train/validation 분리
    val_mask = np.isin(cluster_labels, val_clusters)
    train_mask = ~val_mask
    
    train_df = df_valid[train_mask].reset_index(drop=True)
    val_df = df_valid[val_mask].reset_index(drop=True)
    
    # 7. 결과 요약
    actual_val_ratio = len(val_df) / len(df_valid)
    
    print(f"\n분리 결과:")
    print(f"  전체: {len(df_valid)}개")
    print(f"  Train: {len(train_df)}개 ({len(train_df)/len(df_valid)*100:.1f}%)")
    print(f"  Validation: {len(val_df)}개 ({actual_val_ratio*100:.1f}%)")
    print(f"  Validation 클러스터: {val_clusters}")
    
    # 8. 클러스터 간 유사도 분석
    print("\n클러스터 간 유사도 분석 중...")
    train_clusters = [c for c in unique_clusters if c not in val_clusters]
    
    # 각 클러스터의 중심점 계산
    train_centroids = []
    val_centroids = []
    
    for cluster_id in train_clusters:
        cluster_mask = cluster_labels == cluster_id
        centroid = np.mean(ecfp_matrix[cluster_mask], axis=0)
        train_centroids.append(centroid)
    
    for cluster_id in val_clusters:
        cluster_mask = cluster_labels == cluster_id
        centroid = np.mean(ecfp_matrix[cluster_mask], axis=0)
        val_centroids.append(centroid)
    
    if train_centroids and val_centroids:
        # Tanimoto 유사도 계산 (Jaccard 유사도)
        train_centroids = np.array(train_centroids)
        val_centroids = np.array(val_centroids)
        
        # 이진화 (임계값 0.5)
        train_centroids_bin = (train_centroids > 0.5).astype(int)
        val_centroids_bin = (val_centroids > 0.5).astype(int)
        
        # Jaccard 거리 계산 (1 - Jaccard 유사도)
        distances = pairwise_distances(train_centroids_bin, val_centroids_bin, metric='jaccard')
        min_distance = np.min(distances)
        max_similarity = 1 - min_distance
        
        print(f"  Train-Validation 클러스터 간 최대 Tanimoto 유사도: {max_similarity:.3f}")
        print(f"  Train-Validation 클러스터 간 최소 Jaccard 거리: {min_distance:.3f}")
    
    return train_df, val_df

In [20]:
df = pd.read_pickle("/home/tech/Hawon/dacon-jump-ai-2025/data/250714_preprocessed_HW_V2.pkl")
train_df, val_df = ecfp_cluster_train_val_split(df, val_ratio=0.2, n_clusters=3)

ECFP 피처 생성 중...
유효한 분자 수: 25753
클러스터 수: 3
클러스터링 수행 중...
클러스터 정보:
  클러스터 1: 17776개 분자
  클러스터 0: 5139개 분자
  클러스터 2: 2838개 분자

분리 결과:
  전체: 25753개
  Train: 20614개 (80.0%)
  Validation: 5139개 (20.0%)
  Validation 클러스터: [0]

클러스터 간 유사도 분석 중...
  Train-Validation 클러스터 간 최대 Tanimoto 유사도: 0.444
  Train-Validation 클러스터 간 최소 거리: 0.556


/home/tech/Hawon/dacon-jump-ai-2025/.venv/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2463: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [35]:
split = {}

split["train"] = train_df.medai_id.values.tolist()
split["validation"] = val_df.medai_id.values.tolist()

with open("../data/tr_vl_split_250717.json", "w") as f:
    json.dump(split, f, indent=2)

### Raw data에서 not imputed data 만 스플릿

In [2]:
df = pd.read_pickle("/home/tech/Hawon/dacon-jump-ai-2025/data/250714_preprocessed_HW_V2.pkl")
df.head(5)

,smiles,ic50_nm,ic50_nm_imputed,is_active(10um),data_source,data_id,pvalue,pvalue_imputed,rdmol,rdmol_confs5,medai_id
0,BrC1=C2C(=C(Br)C(Br)=C1Br)N=C(NCCCN)N2,130.000000,130.000000,True,CAS,"N1-(4,5,6,7-Tetrabromo-1H-benzimidazol-2-yl)-1...",6.886057,6.886057,<rdkit.Chem.rdchem.Mol object at 0x7f26d95facf0>,<rdkit.Chem.rdchem.Mol object at 0x7f2635533100>,Medai_00000
1,BrC1=C2C(N=C(NCCCNC(CCC(N[C@H](C(N[C@H](C(N[C@...,25.000000,25.000000,True,CAS,nan,7.602060,7.602060,<rdkit.Chem.rdchem.Mol object at 0x7f26d95fad90>,<rdkit.Chem.rdchem.Mol object at 0x7f26356cb150>,Medai_00001
2,BrC1=C2C=3C(=C(NC(=O)C4=CC(=C(C)C=C4F)N5C=C(N=...,39.800000,39.800000,True,CAS,"N-(1-Bromo-5,6-dihydroimidazo[1,5-d][1,4]benzo...",7.400117,7.400117,<rdkit.Chem.rdchem.Mol object at 0x7f26d95fae30>,<rdkit.Chem.rdchem.Mol object at 0x7f26356cb1a0>,Medai_00002
3,BrC1=CN(C=2C1=CN=C(NC(=O)C3=CC=C([C@@](CO)(C)O...,7.000000,7.000000,True,CAS,"N-(3-Bromo-1-cyclopropyl-1H-pyrrolo[3,2-c]pyri...",8.154902,8.154902,<rdkit.Chem.rdchem.Mol object at 0x7f26d95faed0>,<rdkit.Chem.rdchem.Mol object at 0x7f26356cb1f0>,Medai_00003
4,BrC=1C2=C(N=CN=C2NC1)N3CCC(N(C(=O)C=4C=CN=CC4)...,89.125367,89.125367,True,CAS,"N-[1-(5-Bromo-7H-pyrrolo[2,3-d]pyrimidin-4-yl)...",7.049999,7.049999,<rdkit.Chem.rdchem.Mol object at 0x7f26d95faf70>,<rdkit.Chem.rdchem.Mol object at 0x7f26356cb240>,Medai_00004


In [3]:
df = df.dropna(subset=["ic50_nm"])
train_df, val_df = ecfp_cluster_train_val_split(df, val_ratio=0.1, n_clusters=6)

ECFP 피처 생성 중...
유효한 분자 수: 3913
클러스터 수: 6
클러스터링 수행 중...
클러스터 정보:
  클러스터 0: 1094개 분자
  클러스터 5: 1078개 분자
  클러스터 1: 787개 분자
  클러스터 4: 479개 분자
  클러스터 2: 247개 분자
  클러스터 3: 228개 분자

분리 결과:
  전체: 3913개
  Train: 3666개 (93.7%)
  Validation: 247개 (6.3%)
  Validation 클러스터: [2]

클러스터 간 유사도 분석 중...
  Train-Validation 클러스터 간 최대 Tanimoto 유사도: 0.250
  Train-Validation 클러스터 간 최소 Jaccard 거리: 0.750


/home/tech/Hawon/dacon-jump-ai-2025/.venv/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2463: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [4]:
split = {}

split["train"] = train_df.medai_id.values.tolist()
split["validation"] = val_df.medai_id.values.tolist()

with open("../data/250727_split_noImputed_vlsize0.1.json", "w") as f:
    json.dump(split, f, indent=2)